## Upload File to virtual environment

File will be deleted after you close the window or notebook disconnects


In [1]:
from google.colab import files

# Prompt the user to upload a file
uploaded_files = files.upload()

# Access the uploaded file
for file_name, file_content in uploaded_files.items():
    # Process the uploaded file as needed
    print(f"Uploaded file: {file_name}")
    # Do further processing with file_content


Saving preferences.csv to preferences (3).csv
Uploaded file: preferences (3).csv





## Read CSV preference file




In [2]:
import pandas as pd
import numpy as np
import random

# Read CSV file as DataFrame
data = pd.read_csv('preferences.csv')
data = data.sample(frac=1)
#Shuffle the data to avoid order bias

data= data.sample(frac=1)
# Print the updated DataFrame
data.head()


,student ID,bio,green,solar,theater,LCA
23,L04464374,1,2,3,4,5
5,L04452961,1,5,2,3,4
29,L04457001,1,2,4,5,3
21,L04453589,2,1,3,4,5
0,L04457643,2,4,1,5,3


## Create two tables for selection for Section 1 and 2

In [3]:
#Create Selection DF


column_names = data.columns[1:]

Section1 = pd.DataFrame({
    'student ID': data['student ID'],
    **{column: 0 for column in column_names}
})
Section2 = pd.DataFrame({
    'student ID': data['student ID'],
    **{column: 0 for column in column_names}
})





## Implement Heuristic

Assing 1st module

1.	Go over each student, assign preference 1 to section 1
2.	If that section is full, assign preference 1 to sections 2.
3.	If both sections are full for preference 1, assign preference 2 to sections 1
4.	 If that is full, assign preference 2  to sections 2
5.	If both sections are full for preference 2, assign preference 3 to sections 1
6.	If that is full, assign preference 3 to sections 2


In [4]:
def check_section1_full(section):
   return Section1[section].sum()>=20

def check_section2_full(section):
   return Section2[section].sum()>=20

def check_student_assing_section(id):
    if(Section1.loc[Section1['student ID']==id, ].iloc[:,1:data.shape[1]].sum().sum()==1):
        return 1
    if(Section2.loc[Section2['student ID']==id, ].iloc[:,1:data.shape[1]].sum().sum()==1):
        return 2
    return 0


In [5]:
#for s in range(30):#data.shape[0]
for s in range(data.shape[0]):

    student_prefs=pd.DataFrame(data.iloc[s]).T

    #Pick preference 1
    student_pref_1=student_prefs.columns[student_prefs.iloc[0] == 1][0]
    #Pick preference 2
    student_pref_2=student_prefs.columns[student_prefs.iloc[0] == 2][0]
    #Pick preference 3
    student_pref_3=student_prefs.columns[student_prefs.iloc[0] == 3][0]

    #print(s)

    #Check if preference 1 is full in sections 1
    if(check_section1_full(student_pref_1)):

        #If pref 1 sections 1 is full check section 2
        #Check if preference 1 ections 2 is full
        if(check_section2_full(student_pref_1)):

            #If pref 1 sections 2 is also full, assing pref 2 section 1
            #Check if preference 2 is full in sections 1
            if(check_section1_full(student_pref_2)):

                #If pref 2 sections 1 is also full, assing pref 2 section 2
                #Check if preference 2 is full in sections 2
                if(check_section2_full(student_pref_2)):

                    #If pref 2 sections 2 is also full, assing pref 3 section 1
                    #Check if preference 3 is full in sections 1
                    if(check_section1_full(student_pref_3)):

                        #If pref 3 sections 1 is also full, assing pref 3 section 1
                        #stop checking additional pref
                        #print(student_pref_3+ "sec2")

                        Section2.loc[Section2['student ID']==student_prefs.iloc[0,0], student_pref_3]=1


                    else: #If pref 3 is not full in section 1 assing
                        #print(student_pref_3+ "sec1")

                        Section1.loc[Section1['student ID']==student_prefs.iloc[0,0], student_pref_3]=1


                else: #If pref 2 is not full in section 2 assing
                    #print(student_pref_2+ "sec2")

                    Section2.loc[Section2['student ID']==student_prefs.iloc[0,0], student_pref_2]=1


            else: #If pref 2 is not full in section 1 assing
                #print(student_pref_2+ "sec1")

                Section1.loc[Section1['student ID']==student_prefs.iloc[0,0], student_pref_2]=1


        else: #If preference 1 section 2 is not full, assing
            #print(student_pref_1+ "sec2")

            Section2.loc[Section2['student ID']==student_prefs.iloc[0,0], student_pref_1]=1


    else:  #if preference 1 sections 1 is not full, assign
        #print(student_pref_1+ "sec1")

        Section1.loc[Section1['student ID']==student_prefs.iloc[0,0], student_pref_1]=1



Assing 2nd module

1.	Go over each student, assign preference 2 to section still unassigned

2.	If preferene 2 is full, go for preference 3.



In [6]:
#s=10
#for s in range(1):#data.shape[0]
for s in range(data.shape[0]):

    #print(s)
    student_prefs=pd.DataFrame(data.iloc[s]).T


    #Pick preference 2
    student_pref_2=student_prefs.columns[student_prefs.iloc[0] == 2][0]
    #Pick preference 3
    student_pref_3=student_prefs.columns[student_prefs.iloc[0] == 3][0]

    #Check which module student is missing (section 1 or 2):
    student_id=student_prefs.iloc[0][0]
    assigned=check_student_assing_section(student_id)

    #Assinged sections 1 done, so look at sections 2
    if(assigned==1):

        #Check if pref 2 of sections 2 is full
        if(check_section2_full(student_pref_2)):

            #If pref 2 of sections 2 if full, assign pref 3
            #print(student_pref_3+ "sec2")
            Section2.loc[Section2['student ID']==student_prefs.iloc[0,0], student_pref_3]=1

        #if pref 2 of the  sections 2 is not full assign
        else:
           # print(student_pref_2+ "sec2")
            Section2.loc[Section2['student ID']==student_prefs.iloc[0,0], student_pref_2]=1


    #Assinged sections 2 done, so look at sections 1
    else:

        #Check if pref 2 of sections 1 is full
        if(check_section1_full(student_pref_2)):

            #If pref 2 of sections 1 if full, assign pref 3
            #print(student_pref_3+ "sec1")
            Section1.loc[Section1['student ID']==student_prefs.iloc[0,0], student_pref_3]=1

        #if pref 2 of the  sections 1 is not full assign
        else:
            #print(student_pref_2+ "sec1")
            Section1.loc[Section1['student ID']==student_prefs.iloc[0,0], student_pref_2]=1




## Number of student per module per sections

In [7]:
Sec1_No=Section1.iloc[:, 1:data.shape[1]].sum()
Sec1_No

bio        20
green       7
solar       6
theater     2
LCA         6
dtype: int64

In [8]:
Sec2_No=Section2.iloc[:, 1:data.shape[1]].sum()
Sec2_No

bio        16
green       9
solar       3
theater     7
LCA         6
dtype: int64

In [9]:
def abs_diff_sections(Sec1_No,Sec2_No):
    abs_sum=0
    for i in range(len(Sec1_No)):
        abs_sum+=abs(Sec1_No[i]-Sec2_No[i])
    return abs_sum

abs_diff_sections(Sec1_No,Sec2_No)

14

## Number of sections per student

In [10]:
frequency_table = pd.DataFrame(Section1.iloc[:, 1:data.shape[1]].sum(axis=1).value_counts())
# Reset the index and rename the columns
frequency_table.reset_index(inplace=True)
frequency_table.columns = ['Sec1 assigned', 'No. Students']
frequency_table

,Sec1 assigned,No. Students
0,1,41


In [11]:
frequency_table = pd.DataFrame(Section2.iloc[:, 1:data.shape[1]].sum(axis=1).value_counts())
# Reset the index and rename the columns
frequency_table.reset_index(inplace=True)
frequency_table.columns = ['Sec2 assigned', 'No. Students']
frequency_table

,Sec2 assigned,No. Students
0,1,41


##  Balance student between Sections 1 and 2

1. Go over each student and check if module for section 1 and 2 can be flipped (i.e., no module will go above 20) and if so, would the change improve the absolute student different between sections

In [12]:

for s in range(data.shape[0]):


    #Section 1 pick
    Sec1=pd.DataFrame(Section1.iloc[s]).T
    sec1_pick=Sec1.columns[Sec1.iloc[0] == 1][0]

    #Section 2 pick
    Sec2=pd.DataFrame(Section2.iloc[s]).T
    sec2_pick=Sec2.columns[Sec2.iloc[0] == 1][0]

    #Check if making the change will make a module have more than 20
    if(Sec2_No[sec1_pick]+1<=20 and  Sec1_No[sec2_pick]+1<=20):

        #if making the change will not make a module have more than 20
        #Check if it improve the balance

        #potential distribucion
        Sec1_No_p=Sec1_No.copy()
        Sec2_No_p=Sec2_No.copy()

        Sec1_No_p[sec1_pick]-=1
        Sec2_No_p[sec2_pick]-=1

        Sec1_No_p[sec2_pick]+=1
        Sec2_No_p[sec1_pick]+=1

        if(abs_diff_sections(Sec1_No_p,Sec2_No_p)<abs_diff_sections(Sec1_No,Sec2_No)):

            #If it improves the "balance", make the change
            student_id_to_match=Sec1.iloc[0,0]

            Section1.at[Section1[Section1['student ID'] == student_id_to_match].index[0], sec1_pick] = 0
            Section2.at[Section2[Section2['student ID'] == student_id_to_match].index[0], sec2_pick] = 0

            Section1.at[Section1[Section1['student ID'] == student_id_to_match].index[0], sec2_pick] = 1
            Section2.at[Section2[Section2['student ID'] == student_id_to_match].index[0], sec1_pick] = 1

            #Update distribucion var
            Sec1_No=Section1.iloc[:, 1:data.shape[1]].sum()
            Sec2_No=Section2.iloc[:, 1:data.shape[1]].sum()



In [13]:

for s in range(data.shape[0]):


    #Section 1 pick
    Sec1=pd.DataFrame(Section1.iloc[s]).T
    sec1_pick=Sec1.columns[Sec1.iloc[0] == 1][0]

    #Section 2 pick
    Sec2=pd.DataFrame(Section2.iloc[s]).T
    sec2_pick=Sec2.columns[Sec2.iloc[0] == 1][0]

    #Check if making the change will make a module have more than 20
    if(Sec2_No[sec1_pick]+1<=20 and  Sec1_No[sec2_pick]+1<=20):

        #if making the change will not make a module have more than 20
        #Check if it improve the balance

        #potential distribucion
        Sec1_No_p=Sec1_No.copy()
        Sec2_No_p=Sec2_No.copy()

        Sec1_No_p[sec1_pick]-=1
        Sec2_No_p[sec2_pick]-=1

        Sec1_No_p[sec2_pick]+=1
        Sec2_No_p[sec1_pick]+=1

        if(abs_diff_sections(Sec1_No_p,Sec2_No_p)<abs_diff_sections(Sec1_No,Sec2_No)):

            #If it improves the "balance", make the change
            student_id_to_match=Sec1.iloc[0,0]

            Section1.at[Section1[Section1['student ID'] == student_id_to_match].index[0], sec1_pick] = 0
            Section2.at[Section2[Section2['student ID'] == student_id_to_match].index[0], sec2_pick] = 0

            Section1.at[Section1[Section1['student ID'] == student_id_to_match].index[0], sec2_pick] = 1
            Section2.at[Section2[Section2['student ID'] == student_id_to_match].index[0], sec1_pick] = 1

            #Update distribucion var
            Sec1_No=Section1.iloc[:, 1:data.shape[1]].sum()
            Sec2_No=Section2.iloc[:, 1:data.shape[1]].sum()



## Number of student per module per sections

In [14]:
Sec1_No=Section1.iloc[:, 1:data.shape[1]].sum()
Sec1_No

bio        18
green       8
solar       6
theater     3
LCA         6
dtype: int64

In [15]:
Sec2_No=Section2.iloc[:, 1:data.shape[1]].sum()
Sec2_No

bio        18
green       8
solar       3
theater     6
LCA         6
dtype: int64

In [16]:
abs_diff_sections(Sec1_No,Sec2_No)

6

## Number of sections per student

In [17]:
frequency_table = pd.DataFrame(Section1.iloc[:, 1:data.shape[1]].sum(axis=1).value_counts())
# Reset the index and rename the columns
frequency_table.reset_index(inplace=True)
frequency_table.columns = ['Sec1 assigned', 'No. Students']
frequency_table

,Sec1 assigned,No. Students
0,1,41


In [18]:
frequency_table = pd.DataFrame(Section2.iloc[:, 1:data.shape[1]].sum(axis=1).value_counts())
# Reset the index and rename the columns
frequency_table.reset_index(inplace=True)
frequency_table.columns = ['Sec2 assigned', 'No. Students']
frequency_table

,Sec2 assigned,No. Students
0,1,41


# Generate final output file

In [53]:
data['Section 1']=None
data['Section 2']=None


In [58]:
# Iterate over each row in 'Section1'
for index, row in Section1.iterrows():
    # Filter the row values equal to 1

    columns_with_1 = row[row == 1].index.tolist()
    data.at[index,'Section 1']=columns_with_1[0]


# Iterate over each row in 'Section2'
for index, row in Section2.iterrows():
    # Filter the row values equal to 1

    columns_with_1 = row[row == 1].index.tolist()
    data.at[index,'Section 2']=columns_with_1[0]

In [61]:
from google.colab import files
# Download the file
data.to_csv('ES101_assignment.csv', index=False)
files.download('ES101_assignment.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>